In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
df = pd.read_csv("SQL_Capstone2.csv")

In [64]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [65]:
attrition_data = [
    ['True', (df['Attrition'] == 'Yes').mean() * 100],
    ['False', (df['Attrition'] == 'No').mean() * 100]
]

In [66]:
attrition_df = pd.DataFrame(attrition_data, columns = ['Attrition','Attrition_Rate'])

In [67]:
attrition_df

,Attrition,Attrition_Rate
0,True,16.122449
1,False,83.877551


This is the data distribution of the attrition in the dataset.

In [68]:
total_emp = len(df)

In [69]:
attrition_gender = df.groupby(['Attrition','Gender']).size().reset_index(name='Count_Gender')

In [70]:
attrition_gender['Attrition_by_Gender'] = round(attrition_gender['Count_Gender'] / total_emp * 100,2)

In [71]:
attrition_gender

,Attrition,Gender,Count_Gender,Attrition_by_Gender
0,No,Female,501,34.08
1,No,Male,732,49.80
2,Yes,Female,87,5.92
3,Yes,Male,150,10.20


The above dataframe shows the distribution of attrition based on the genders.

In [72]:
attrition_department = df.groupby(['Attrition','Department']).size().reset_index(name='Count_Department')
#attrition_department['Attrition_by_Department'] = attrition_department['Count_Department'] / total_emp * 100
attrition_department

,Attrition,Department,Count_Department
0,No,Human Resources,51
1,No,Research & Development,828
2,No,Sales,354
3,Yes,Human Resources,12
4,Yes,Research & Development,133
5,Yes,Sales,92


The above dataframe created has the count of each department distribution for attrition.

In [73]:
def age_group(age):
    if age < 30:
        return 'Under 30'
    elif age < 40:
        return '30-40'
    elif age < 50:
        return '40-50'
    else:
        return 'Over 50'

df['Age_Group'] = df['Age'].apply(age_group)

In [74]:
attrition_agegroup = df.groupby(['Attrition','Age_Group']).size().reset_index(name='num')
attrition_agegroup['percent_by_age'] = round(attrition_agegroup['num'] / total_emp * 100,2)
attrition_agegroup

,Attrition,Age_Group,num,percent_by_age
0,No,30-40,533,36.26
1,No,40-50,315,21.43
2,No,Over 50,150,10.20
3,No,Under 30,235,15.99
4,Yes,30-40,89,6.05
5,Yes,40-50,34,2.31
6,Yes,Over 50,23,1.56
7,Yes,Under 30,91,6.19


The above dataframe shows us the percentage of each age group in the each attrition category.

In [75]:
df['YearsAtCompany']

0        6
1       10
2        0
3        8
4        2
        ..
1465     5
1466     7
1467     6
1468     9
1469     4
Name: YearsAtCompany, Length: 1470, dtype: int64

In [76]:
def tenure(Years):
    if Years < 2:
        return 'New Hires'
    elif Years <= 5 and Years >= 2:
        return '2-5 Years'
    elif Years <= 10 and Years >= 6:
        return '6-10 Years'
    elif Years <= 20 and Years >= 11:
        return '11-20 Years'
    else:
        return 'Over 20'

df['Tenure'] = df['YearsAtCompany'].apply(tenure)

In [77]:
#attrition_tenure = df[df['Attrition'] == True]
attrition_tenure = df.groupby(['Attrition','Tenure']).size().reset_index(name='num')
attrition_tenure['percent'] = round(attrition_tenure['num'] / total_emp * 100,2)
attrition_true = attrition_tenure[attrition_tenure['Attrition'] == 'Yes']

In [78]:
attrition_true

,Attrition,Tenure,num,percent
5,Yes,11-20 Years,12,0.82
6,Yes,2-5 Years,87,5.92
7,Yes,6-10 Years,55,3.74
8,Yes,New Hires,75,5.10
9,Yes,Over 20,8,0.54


The above dataframe shows us the number and percentage of total employees who falls in a specific tenure of working with the company and has the attrition value as True.

In [121]:
import sqlite3

In [122]:
df_sql = pd.read_csv("SQL_Capstone2.csv")

In [123]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("hr_data.db")
df_sql.to_sql(name='hr_data',con=conn,if_exists='replace',index=False)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Available tables:", tables)
conn.close()

Available tables: [('company_data',), ('hr_data',)]


Looking fot the avaliable tables in the databses. There are 2 tables. One is Company_data and otehr is hr_data. hr_data is the one that we require.

In [124]:
import sqlite3
conn = sqlite3.connect("employee_data.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS company_data;")
conn.commit()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
conn.close()


[('hr_data',)]


We dont need company_data so I have deleted the company_data for saving the space.

In [128]:
conn = sqlite3.connect("employee_data.db")

# First query
query1 = """
SELECT 
    CASE 
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
        WHEN Age > 50 THEN 'Over 50'
    END AS age_group,
    JobRole AS role,
    WorkLifeBalance AS wlb,
    JobSatisfaction AS job_sat,
    COUNT(*) AS total_emp,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS attr_count,
    ROUND(100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS attr_rate
FROM hr_data
WHERE Age >= 40
GROUP BY age_group, role, wlb, job_sat
ORDER BY age_group, attr_rate DESC;
"""

df1 = pd.read_sql_query(query1, conn)
print(df1)

  age_group                role  wlb  job_sat  total_emp  attr_count  \
0     40-50     Sales Executive    1        4          1           1   
1     40-50  Research Scientist    3        2          1           0   

   attr_rate  
0      100.0  
1        0.0  


So, the above query is to fetch the data of employees above 40 who is part of attrition. If you look at the output data there are no employees above 50 who have attrition as true. So, even in the range of 40-50 there are 2 employees. One of them has less work life balance and the otehr has more work life balance. Even thought the employee with higher job satisfaction he was part of attrition. So, that means the work life balance places a important role in  attrition.

In [129]:
conn = sqlite3.connect("employee_data.db")
query2 = """
SELECT 
    CASE 
        WHEN MonthlyIncome < 5000 THEN 'Low Income (<5k)'
        WHEN MonthlyIncome BETWEEN 5000 AND 10000 THEN 'Mid Income (5k-10k)'
        ELSE 'High Income (>10k)'
    END AS IncomeLevel,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND(100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS AttritionRate
FROM hr_data
GROUP BY IncomeLevel
ORDER BY AttritionRate DESC;
"""

df2 = pd.read_sql_query(query2, conn)
print(df2)

           IncomeLevel  TotalEmployees  AttritionCount  AttritionRate
0  Mid Income (5k-10k)               2               1          50.00
1     Low Income (<5k)               3               1          33.33


In [131]:
conn = sqlite3.connect("employee_data.db")
query3 = """
SELECT JobRole, OverTime, WorkLifeBalance, JobSatisfaction,
       COUNT(*) AS TotalEmployees,
       SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount
FROM hr_data
WHERE MonthlyIncome > 10000
GROUP BY JobRole, OverTime, WorkLifeBalance, JobSatisfaction
ORDER BY AttritionCount DESC;
"""

df3 = pd.read_sql_query(query3, conn)
print(df3)

Empty DataFrame
Columns: [JobRole, OverTime, WorkLifeBalance, JobSatisfaction, TotalEmployees, AttritionCount]
Index: []


After observing to the outputs of above 2 queries. So, the salary range is playing a vital role in the attrition. In low and mid scaled salary range. There are attrition but not in the high salary range which is 10000+. So, that means if the employees are satified with their salary they are leaving the company.

In [134]:
conn = sqlite3.connect("employee_data.db")
query4 = """
SELECT 
    JobRole,
    Department,
    OverTime,
    TrainingTimesLastYear,
    WorkLifeBalance,
    COUNT(*) AS TotalEmployees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS AttritionCount,
    ROUND(100.0 * SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS AttritionRate
FROM hr_data
WHERE YearsAtCompany < 5
GROUP BY JobRole, Department, OverTime, TrainingTimesLastYear, WorkLifeBalance
ORDER BY AttritionRate DESC;
"""

df4 = pd.read_sql_query(query4, conn)
print(df4)



                 JobRole              Department OverTime  \
0  Laboratory Technician  Research & Development      Yes   
1  Laboratory Technician  Research & Development       No   

   TrainingTimesLastYear  WorkLifeBalance  TotalEmployees  AttritionCount  \
0                      3                3               1               1   
1                      3                3               1               0   

   AttritionRate  
0          100.0  
1            0.0  


The above query is to check for the employees who have left the company within 5 years arranged by each role, depatment and other features. So the person who is doing overtime has left the compnay within 5 years. So, this means doing overtime they might have been exhausted that made them leave the company.